# Rx 数据流

RxJs一切都已数据流为中心

与数据流相关操作符（绝大部分为静态操作符） -- 任何一个操作符都会返回Observable对象，创建类操作符不依赖于其他Observable对象

## 同步数据流

Rx以同步的方式进行吞吐数据，数据之间不存在时间间隔

包括函数`create`、`of`、`range`、`generate`、`repeat`、`empty`、`throw`、`never`

In [ ]:
import { Observable } from 'rxjs/Observable'
var source$ = Observable.create(observer => {
    observer.next(1);
    observer.next(2);
    observer.next(3);
});
var subscription = source$.subscribe(console.log);
import 'rxjs/add/observable/of'
// of 操作符 列举数据
var source$ = Observable.of(1, 2, 3);
var subscription = source$.subscribe(console.log)

In [ ]:
import { Observable } from 'rxjs/Observable'
// 指定范围，step值固定为1
import 'rxjs/add/observable/range';
// 创建循环
import 'rxjs/add/observable/generate';
var source$ = Observable.range(1, 40);
// var subscription = source$.subscribe(console.log);
var source1$ = Observable.generate(
    0, // 初始值
    value => value < 100, // 循环条件
    value => value + 3, // step
    value => value ** 2 // map
)
var subscription1 = source1$.subscribe(console.log);

In [ ]:
// 重复数据流
import { Observable } from 'rxjs/Observable';
import 'rxjs/add/observable/of';
import 'rxjs/add/operator/repeat';
var source$ = Observable.of(1, 2, 3);
var repeat$ = source$.repeat(2);
// repeat 重复订阅n次上游Observable对象n次
// repeat只有在上游Observable对象完结之后才重新订阅，如果上游Observable对象永不完结，
var subscription = repeat$.subscribe(console.log);

// 用generate实现repeat
import 'rxjs/add/observable/generate';
source$ = Observable.generate(
    0,
    value => value < 30,
    value => value + 1,
    value => (value % 3) + 1 
);
subscription = source$.subscribe(console.log);

## 创建异步数据流
即**异步Observable**对象


In [ ]:
$$.async();
// interval 定时产生数据
import { Observable } from 'rxjs/Observable';
import 'rxjs/add/observable/interval';

var source$ = Observable.interval(1000); // 每1000ms产生一个数据
var subscription = source$.subscribe(value => console.log(`interval函数生成：${value}`));

(function($$) {
    setTimeout(() => {
        subscription.unsubscribe();
        // 告诉当前cell 异步执行已经完成
        $$.done();
    }, 1000 * 10);
})($$);

// timer 定时产生数据
import 'rxjs/add/observable/timer';
// 注意，timer函数还有第二个参数，如果设置了第二个参数则功能类似于interval
// 参数1与参数2一致的话则
var source1$ = Observable.timer(2000);
var subscription1 = source1$.subscribe(value => {
    console.log(`timer函数生成：${value}`);
    subscription1.unsubscribe();
})

In [ ]:
$$.async();
// 将其他操作转化为异步Observable对象
import { Observable } from 'rxjs/Observable';
import { get } from 'axios';
import 'rxjs/add/observable/from';
// form可以接受的参数类型：数字，字符串，数字，类数组，promise，generator
function toObservable() {
    return Observable.from(arguments);
}
var source1$ = toObservable(1, 2, 3, 4);
source1$.subscribe(console.log);

// promise
var source$ = Observable.from(get('http://poetry.apiopen.top/getTime'));
source$.subscribe(res => {
    console.log(res);
});
import 'rxjs/add/observable/fromPromise';
var source2$ = Observable.fromPromise(Promise.reject("aaaaa"));
source2$.subscribe(console.log, console.error);

// event
import EventEmitter from 'events';
import 'rxjs/add/observable/fromEvent';
var customEvent = new EventEmitter();

var eventSource$ = Observable.fromEvent(customEvent, 'notify');
eventSource$.subscribe((event) => {
    console.log(event);
});
setTimeout(() => {
    customEvent.emit('notify', '123', '456');
    customEvent.emit('notify', 'hello');
}, 0);
// 自定义事件源
import 'rxjs/add/observable/fromEventPattern';
var customEvent = new EventEmitter();

var addHandler = (handler) => {
    customEvent.addListener('notify', handler);
};
var removeHandler = (handler) => {
    customEvent.removeListener('notify', handler);
};
var customEventSource$ = Observable.fromEventPattern(addHandler, removeHandler);
customEventSource$.subscribe(console.log);
setTimeout(() => {
    customEvent.emit('notify', 'custom event');
}, 0);

// ajax 浏览器api 注意导入路径
// import 'rxjs/add/observable/dom/ajax';
// var ajaxSource$ = Observable.ajax('http://poetry.apiopen.top/getTime', {responseType: 'json'})
// ajaxSource$.subscribe(console.log);

(function($$) {
    setTimeout(() => {
        $$.done();
    }, 5000);
})($$);

In [3]:
import { Observable } from 'rxjs/Observable';
import 'rxjs/add/operator/repeatWhen';
import 'rxjs/add/observable/timer';
import 'rxjs/add/observable/interval';
import 'rxjs/add/observable/of';
import 'rxjs/add/operator/delay';

// 重复订阅，与repeat相比可以指定订阅时间
var notify = notification$ => {
    // 延迟2s后再次订阅
//     return Observable.timer(2000);
    // 每隔1s订阅一次
//     return Observable.interval(1000);
    
    return notification$.delay(2000);
}
var repeatWhenSource$ = Observable.of(1, 2, 3).repeatWhen(notify);
var repeatWhenSubscription = repeatWhenSource$.subscribe(value => {
    console.log(value);
});

(function($$) {
    setTimeout(() => {
        console.log(1111)
        repeatWhenSubscription.unsubscribe();
        $$.done();
    }, 5000);
})($$);

1
2
3
1
2
3
1
2
3
1111


In [5]:
import { Observable } from 'rxjs/Observable';
import 'rxjs/add/observable/of';
import 'rxjs/add/observable/defer';

// 延迟创建Observable对象，等到observer真正被订阅时在创建
var deferObservableFactory = () => Observable.of(1, 2, 3);
var deferObservable$ = Observable.defer(deferObservableFactory);
var deferObservableSubscription = deferObservable$.subscribe(value => {
    console.log(value);
    deferObservableSubscription.unsubscribe();
    $$.done();
});

1
2
3
